# Exploring forward and inverse problems with DC resistivity 

This notebook sets up and runs forward simulations and an inversion for a model with two blocks. You can alter the geometry and physical properties of the blocks and explore the impacts on the data and the inversion result

In [ ]:
# core python 
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets

# SimPEG inversion machinery
from simpeg import maps, utils

# linear solver
from simpeg.utils.solver_utils import get_default_solver
Solver = get_default_solver()

# DC resistivity modules
from simpeg.electromagnetics import resistivity as dc

In [ ]:
from dc_simulation_utils import create_survey, build_mesh, plot_model, plot_data_target_and_background, create_inversion, plot_inversion_results

# Define the survey, model and mesh

In [ ]:
# Generate source list for DC survey line
survey = create_survey(
    survey_type="dipole-dipole", 
    end_points=np.r_[-350.0, 450.0], 
    station_spacing=50.0, 
    num_rx_per_src=8,
)

In [ ]:
mesh, core_domain_x, core_domain_z = build_mesh(survey)
mesh.plot_grid()

# build our model 

You can change the physical properties and geometry of the blocks

In [ ]:
# define the resistivities
rho_background = 100
rho_resistive_block = 1000
rho_conductive_block = 10

# define the geometry of each block
xlim_resistive_block = np.r_[-200, -90]
zlim_resistive_block = np.r_[-100, -50]

xlim_conductive_block = np.r_[200, 90]
zlim_conductive_block = np.r_[-100, -50]

In [ ]:
background_model = rho_background * np.ones(mesh.nC)
resistivity_model = background_model.copy()

resistivity_model = utils.model_builder.add_block(
    mesh.cell_centers, resistivity_model, 
    np.r_[xlim_resistive_block.min(), zlim_resistive_block.min()], np.r_[xlim_resistive_block.max(), zlim_resistive_block.max()],
    rho_resistive_block
)
resistivity_model = utils.model_builder.add_block(
    mesh.cell_centers, resistivity_model, 
    np.r_[xlim_conductive_block.min(), zlim_conductive_block.min()], np.r_[xlim_conductive_block.max(), zlim_conductive_block.max()],
    rho_conductive_block
)

In [ ]:
ax = plot_model(mesh, resistivity_model, core_domain_x, core_domain_z)
ax.set_ylim(np.r_[-300, 0])

## Set up and run simulations

In [ ]:
noise_level = 0.03  # noise level that we assign to the data

In [ ]:
# here, we formulate the simulation in terms of log resistivity
# that way, we enforce that resistivity is positive when we get to the inversion

simulation_dc = dc.Simulation2DNodal(
    mesh, rhoMap=maps.ExpMap(mesh), 
    survey=survey, solver=Solver, storeJ=True
)

In [ ]:
# run a simulation for the background data
background_data = simulation_dc.make_synthetic_data(
    np.log(background_model), relative_error=noise_level, add_noise=True
)
# run a simulation for the model with 2 blocks
target_data = simulation_dc.make_synthetic_data(
    np.log(resistivity_model), relative_error=noise_level, add_noise=True
)

In [ ]:
plot_data_target_and_background(
    target_data, background_data
)

# Set up and run the inversion

In [ ]:
estimated_relative_error = 0.03  # assume 3% uncertainties on the data 
reference_resistivity = rho_background  # starting and reference model 

alpha_s=1e-2  # weight for the smallness term in the regularization
alpha_x=1  # weight for the smoothness in the x-direction
alpha_y=1  # weight for the smoothness in the y-direction

In [ ]:
inv, target_misfit, inversion_log = create_inversion(
    simulation_dc,
    target_data,
    reference_resistivity=reference_resistivity,
    relative_error=estimated_relative_error,
    alpha_s=alpha_s, 
    alpha_x=alpha_x, 
    alpha_y=alpha_y,
    maxIter=20,
    beta0_ratio=1e2,
    beta_cooling_factor=2,
    beta_cooling_rate=1,
)

In [ ]:
starting_model = np.log(rho_background) * np.ones(mesh.n_cells)
model_recovered = inv.run(starting_model)

In [ ]:
maxiter = len(inversion_log.outDict)-1
def plot_inversion_results_interactive(iteration):
    plot_inversion_results(
        simulation_dc, target_data, inversion_log, rho_background, 
        core_domain_x, core_domain_z, iteration,
    );

In [ ]:
ipywidgets.interact(
    plot_inversion_results_interactive,
    iteration=ipywidgets.IntSlider(min=1, max=maxiter, value=1)
);